In [1]:
class Variable:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return self.name

class Constant:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return str(self.value)

class Function:
    def __init__(self, name, args):
        self.name = name
        self.args = args

    def __repr__(self):
        return f"{self.name}({', '.join(map(str, self.args))})"


def is_variable(term):
    return isinstance(term, Variable)

def is_constant(term):
    return isinstance(term, Constant)

def is_function(term):
    return isinstance(term, Function)

def function_name(term):
    return term.name

def function_arguments(term):
    return term.args



def occurs_check(var, term):
    if var == term:
        return True
    if is_variable(term):
        return False
    if is_function(term):
        return any(occurs_check(var, arg) for arg in function_arguments(term))
    return False



def unify(t1, t2, substitution=None):
    if substitution is None:
        substitution = {}


    if t1 == t2:
        return substitution


    elif is_variable(t1):
        if occurs_check(t1, t2):
            return None  # Failure
        return apply_substitution({t1: t2}, substitution)


    elif is_variable(t2):
        if occurs_check(t2, t1):
            return None  # Failure
        return apply_substitution({t2: t1}, substitution)


    elif is_constant(t1) and is_constant(t2):
        if t1 != t2:
            return None  # Failure
        return substitution


    elif is_function(t1) and is_function(t2):
        if function_name(t1) != function_name(t2):
            return None  # Failure: Different function names

        for arg1, arg2 in zip(function_arguments(t1), function_arguments(t2)):
            substitution = unify(arg1, arg2, substitution)
            if substitution is None:
                return None  # Failure: Argument unification failed
        return substitution

    return None



def apply_substitution(substitution, current_substitution):

    new_substitution = current_substitution.copy()
    for var, term in substitution.items():

        if var in new_substitution:
            term = apply_substitution({var: term}, new_substitution)
        new_substitution[var] = term
    return new_substitution



def test_unification():
    # Example 1: Simple variable substitution
    X = Variable('X')
    a = Constant('a')
    result = unify(X, a)
    print(f"Unification of X and a: {result}")

    # Example 2: Function unification
    X = Variable('X')
    Y = Variable('Y')
    f = Function('f', [X, a])
    g = Function('g', [Y, a])
    result = unify(f, g)
    print(f"Unification of f(X, a) and g(Y, a): {result}")

    # Example 3: Recursive function unification
    f1 = Function('f', [X, Y])
    f2 = Function('f', [a, a])
    result = unify(f1, f2)
    print(f"Unification of f(X, Y) and f(a, a): {result}")

    # Example 4: Unification failure (different functions)
    h = Function('h', [X])
    result = unify(f1, h)
    print(f"Unification of f(X, Y) and h(X): {result}")


# Run the test
test_unification()



Unification of X and a: {X: a}
Unification of f(X, a) and g(Y, a): None
Unification of f(X, Y) and f(a, a): {X: a, Y: a}
Unification of f(X, Y) and h(X): None
